In [1]:
import torch
import os
import numpy as np

In [2]:
one_train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
one_eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')
two_train_dir = os.path.join('dataset', 'part_two_dataset', 'train_data')
two_eval_dir = os.path.join('dataset', 'part_two_dataset', 'eval_data')

In [3]:
# domains = [{} for _ in range(20)]
# eval_domains = [{} for _ in range(20)]

# for i in range(10):
#     domains[i] = torch.load(os.path.join(one_train_dir, f'{i+1}_train_data.tar.pth'), weights_only=False)
#     domains[i+10] = torch.load(os.path.join(two_train_dir, f'{i+1}_train_data.tar.pth'), weights_only=False)
#     eval_domains[i] = torch.load(os.path.join(one_eval_dir, f'{i+1}_eval_data.tar.pth'), weights_only=False)
#     eval_domains[i+10] = torch.load(os.path.join(two_eval_dir, f'{i+1}_eval_data.tar.pth'), weights_only=False)
    
#     domains[i]['data'] = domains[i]['data'].reshape(2500, -1)
#     domains[i+10]['data'] = domains[i+10]['data'].reshape(2500, -1)
#     eval_domains[i]['data'] = eval_domains[i]['data'].reshape(2500, -1)
#     eval_domains[i+10]['data'] = eval_domains[i+10]['data'].reshape(2500, -1)

In [4]:
train_paths = [os.path.join(one_train_dir, f'{i+1}_train_data.tar.pth') for i in range(10)] + [os.path.join(two_train_dir, f'{i+1}_train_data.tar.pth') for i in range(10)]
eval_paths = [os.path.join(one_eval_dir, f'{i+1}_eval_data.tar.pth') for i in range(10)] + [os.path.join(two_eval_dir, f'{i+1}_eval_data.tar.pth') for i in range(10)]

eval_domains = [torch.load(eval_paths[i], weights_only=False) for i in range(20)]

## Model

In [5]:
from lwp import LWP

## Training and Predictions

In [6]:
from sklearn.metrics import accuracy_score
import pandas as pd

model = LWP(distance_metric='manhattan')

df = pd.DataFrame()

for idx,train_path in enumerate(train_paths):
    
    domain = torch.load(train_path, weights_only=False)    
    x_test = domain['data'].reshape(2500, -1)
    y_pred = model.predict(x_test) if 'targets' not in domain else domain['targets']
    
    model.fit(x_test, y_pred)
    print(model.class_counts)
    del domain
    
    scores = []
    for eval_domain in eval_domains[:idx+1]:
        
        features = eval_domain['data'].reshape(2500, -1)
        labels = eval_domain['targets']
        
        preds = model.predict(features)
        acc = accuracy_score(labels, preds)
        
        scores.append(acc)
        
    df[f'Domain {idx+1}'] = scores + [np.nan] * (20 - len(scores))

{0: 253, 1: 243, 2: 255, 3: 244, 4: 262, 5: 236, 6: 250, 7: 253, 8: 254, 9: 250}
{0: 710, 1: 370, 2: 373, 3: 287, 4: 412, 5: 496, 6: 928, 7: 360, 8: 447, 9: 617}
{0: 1155, 1: 516, 2: 543, 3: 343, 4: 622, 5: 711, 6: 1576, 7: 483, 8: 657, 9: 894}
{0: 1531, 1: 700, 2: 716, 3: 420, 4: 829, 5: 925, 6: 2195, 7: 618, 8: 847, 9: 1219}
{0: 1918, 1: 888, 2: 882, 3: 507, 4: 1056, 5: 1101, 6: 2815, 7: 773, 8: 1032, 9: 1528}
{0: 2316, 1: 1111, 2: 1056, 3: 619, 4: 1262, 5: 1286, 6: 3415, 7: 938, 8: 1208, 9: 1789}
{0: 2713, 1: 1311, 2: 1252, 3: 741, 4: 1471, 5: 1485, 6: 3979, 7: 1089, 8: 1381, 9: 2078}
{0: 3071, 1: 1512, 2: 1449, 3: 880, 4: 1673, 5: 1673, 6: 4552, 7: 1263, 8: 1561, 9: 2366}
{0: 3420, 1: 1731, 2: 1636, 3: 1028, 4: 1894, 5: 1868, 6: 5094, 7: 1411, 8: 1747, 9: 2671}
{0: 3770, 1: 1957, 2: 1842, 3: 1162, 4: 2118, 5: 2049, 6: 5640, 7: 1579, 8: 1942, 9: 2941}
{0: 4144, 1: 2153, 2: 2040, 3: 1332, 4: 2303, 5: 2204, 6: 6165, 7: 1769, 8: 2157, 9: 3233}
{0: 4521, 1: 2348, 2: 2232, 3: 1497, 4: 25

## Evaluation

In [7]:
df

,Domain 1,Domain 2,Domain 3,Domain 4,Domain 5,Domain 6,Domain 7,Domain 8,Domain 9,Domain 10,Domain 11,Domain 12,Domain 13,Domain 14,Domain 15,Domain 16,Domain 17,Domain 18,Domain 19,Domain 20
0,0.2732,0.2564,0.2544,0.2516,0.2472,0.2428,0.2448,0.2428,0.2420,0.2396,0.2396,0.2388,0.2352,0.2336,0.2316,0.2320,0.2336,0.2320,0.2332,0.2336
1,NaN,0.2380,0.2320,0.2320,0.2328,0.2316,0.2312,0.2308,0.2292,0.2284,0.2280,0.2268,0.2256,0.2240,0.2240,0.2244,0.2240,0.2236,0.2224,0.2200
2,NaN,NaN,0.2316,0.2300,0.2300,0.2284,0.2284,0.2280,0.2256,0.2240,0.2240,0.2208,0.2212,0.2192,0.2200,0.2196,0.2184,0.2172,0.2168,0.2184
3,NaN,NaN,NaN,0.2348,0.2308,0.2308,0.2276,0.2260,0.2264,0.2260,0.2264,0.2248,0.2268,0.2240,0.2248,0.2248,0.2260,0.2240,0.2224,0.2224
4,NaN,NaN,NaN,NaN,0.2688,0.2668,0.2648,0.2648,0.2636,0.2624,0.2628,0.2604,0.2600,0.2592,0.2572,0.2548,0.2544,0.2548,0.2540,0.2508
5,NaN,NaN,NaN,NaN,NaN,0.2300,0.2284,0.2260,0.2240,0.2248,0.2232,0.2184,0.2196,0.2196,0.2200,0.2192,0.2192,0.2188,0.2184,0.2172
6,NaN,NaN,NaN,NaN,NaN,NaN,0.2280,0.2256,0.2248,0.2240,0.2224,0.2228,0.2204,0.2192,0.2168,0.2160,0.2148,0.2128,0.2120,0.2116
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2440,0.2424,0.2428,0.2416,0.2416,0.2404,0.2404,0.2392,0.2368,0.2360,0.2336,0.2332,0.2336
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2288,0.2304,0.2320,0.2320,0.2300,0.2308,0.2284,0.2268,0.2276,0.2264,0.2256,0.2252
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2272,0.2248,0.2244,0.2248,0.2244,0.2228,0.2212,0.2208,0.2216,0.2204,0.2204
